<a href="https://colab.research.google.com/github/JSSchouten/TM10007_Group_10/blob/master/TM10007_resit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

# Code

In [0]:
'''
all necessary imports
'''
# general functions
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from pathlib import Path

# load data
from brats.load_data  import load_data

# preprocessing and scaling
from sklearn.model_selection    import train_test_split
from sklearn                    import preprocessing
from sklearn.decomposition      import PCA
from sklearn.feature_selection  import SelectKBest, f_classif

# classifiers
from sklearn.model_selection        import cross_val_score, learning_curve
from sklearn.neighbors              import KNeighborsClassifier
from sklearn.ensemble               import RandomForestClassifier
from sklearn                        import svm

# calculate accuracy values
from sklearn.metrics  import accuracy_score
from sklearn.metrics  import confusion_matrix
from sklearn.metrics  import roc_auc_score

In [0]:
'''
Load the data from GitHub
'''
data = load_data()
data_columns = list(set(data))
# print(f'The number of samples: {len(data.index)}')
# print(f'The number of columns: {len(data.columns)}')

In [0]:
'''
General functions
'''


def split(data):
  '''
  Divide data in a training and test set 80% - 20%
  '''
  x = data.iloc[:,:-1]
  y = data['label']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

  return x_train, x_test, y_train, y_test


def delnan(x_train, x_test):
  '''
  Replace all values causing errors by NaN, and replace those and pre-
  existing NaNs by the column's median
  '''
  data_columns_train = list(set(x_train))
  data_columns_test = list(set(x_test))
  ## Replace inf en -inf by NaN
  x_inf_train = x_train.replace([np.inf, -np.inf], np.nan)
  x_inf_test = x_test.replace([np.inf, -np.inf], np.nan)
  ## Replace strings by NaN
  x_str_train = (x_inf_train.drop(data_columns_train, axis=1)
             .join(x_inf_train[data_columns_train].apply(pd.to_numeric, errors='coerce')))
  x_str_test = (x_inf_test.drop(data_columns_test, axis=1)
             .join(x_inf_test[data_columns_test].apply(pd.to_numeric, errors='coerce')))
  ## Delete all columns containing over 50% NaN
  x_del_nan_train = x_str_train.dropna(axis='columns', thresh= round(0.5 * len(x_str_train)))
  x_del_nan_test = x_str_test.dropna(axis='columns', thresh= round(0.5 * len(x_str_test)))
  ## Replace all NaNs with the trainings set column's median
  x_finished_train = x_del_nan_train.fillna(x_del_nan_train.median())
  x_finished_test = x_del_nan_test.fillna(x_del_nan_train.median())
  
  return x_finished_train, x_finished_test


def standardscaler(x_train, x_test):
  '''
  Scale all values using standard scaling
  '''
  ## Design scaler
  scaler = preprocessing.StandardScaler()
  # scaler = preprocessing.RobustScaler(quantile_range=[5, 95])
  scaler.fit(x_train)

  ## Apply scaler to both sets and return scaled sets
  x_scaled_train = scaler.transform(x_train)
  x_scaled_test = scaler.transform(x_test)

  x_scaled_df_train = pd.DataFrame(x_scaled_train, columns=x_train.columns)
  x_scaled_df_test = pd.DataFrame(x_scaled_test, columns=x_test.columns)

  return x_scaled_df_train, x_scaled_df_test

def pair_plot(x,y,features):
  '''
  Plot the selected features using a pairplot
  '''
  x_pairplot = pd.DataFrame(x)
  x_pairplot.columns = features
  y_pairplot = pd.DataFrame(y, columns=['label'])
  y_pairplot = y_pairplot.reset_index(drop=True)
  total = pd.concat([x_pairplot, y_pairplot], axis=1)
  # fig = seaborn.pairplot(total, hue='label')


def print_result(result, feature, multiply):
  '''
  Print the result dataframe
  Print the prevalence of each classifier in the result dataframe
  Print the mean accuracy of the overall machine learning algorithm
  --------------
  print the features used in the cross validation
  '''
  print('Results'+'-'*80+'\n')
  print(result)
  print('='*80+'\n'+'Results over all iterations:')
  print('Mean AUC:',result['Area Under the Curve'].mean(), 
        '(min:', result['Area Under the Curve'].min(), 'max:', 
        result['Area Under the Curve'].max(), ')')
  print('Sensitivity:',result['Sensitivity'].mean(),
        '(min:', result['Sensitivity'].min(), 'max:', 
        result['Sensitivity'].max(), ')')
  print('Specificity:',result['Specificity'].mean(), 
        '(min:', result['Specificity'].min(), 'max:', 
        result['Specificity'].max(), ')')
  print('='*80)
  if multiply[0].upper() == 'UNIVARIATE':
    print('Prevalence of selected features:'+'\n'+'-'*80+'\n')
    print(feature['Feature'].value_counts(normalize=True)*100*multiply[1])


In [0]:
'''
Model optimization functions
'''


def classif_hyperpar(x_train, y_train, feat_name):
  optim_classif = pd.DataFrame(columns=['clf_name', feat_name])
  train_auc = []
  classif_name = []

  ## Random Forest classifier optimization
  number_of_trees = [10, 30, 100, 200]
  for tree in number_of_trees:
    clf = RandomForestClassifier(n_estimators=tree)
    scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc')
    mean_scores = scores.mean()
    train_auc.append(mean_scores)
    classif_name.append(f'RF {tree}')
    
  ## SVM classifier optimization
  slacks = [0.3, 0.1, 0.05]
  kernels = ['linear', 'rbf', 'poly']
  for kern in kernels:
    for slack in slacks:
      clf = svm.SVC(C= slack, kernel= kern)
      scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc')
      mean_scores = scores.mean()
      train_auc.append(mean_scores)
      classif_name.append(f'SVM {kern} {slack}')

  ## k-nearest neighbors classifier optimization
  number_of_neighbors = [3, 7, 11, 15]
  for neighbor in number_of_neighbors:
    clf = KNeighborsClassifier(n_neighbors= neighbor)
    scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc')
    mean_scores = scores.mean()
    train_auc.append(mean_scores)
    classif_name.append(f'KNN {neighbor}')

  optim_classif['clf_name'] = classif_name
  optim_classif[feat_name] = train_auc

  return optim_classif

  
def features_hyperpar(x_train, y_train):
  '''
  Select discriminating features using PCA & univariate feature selection 
  '''
  data_optim = pd.DataFrame()

  ## PCA feature selection optimization
  pca = PCA(n_components=None)
  pca.fit(x_train)

  variancelist = np.cumsum(pca.explained_variance_ratio_)

  ## Thresholds used in gridsearch
  thresholds = [0.5, 0.75, 0.9, 0.95]
  for thres in thresholds:
    comp = np.searchsorted(variancelist, thres)

    pca_spec = PCA(n_components=comp)
    pca_spec.fit(x_train)
  
    ## Use PCA for all different classifiers to be optimized
    x_pca_train = pd.DataFrame(pca_spec.transform(x_train))
    PCA_name = f'PCA {thres}'
    PCA_classif = classif_hyperpar(x_pca_train, y_train, PCA_name)
    if data_optim.empty:
      data_optim = PCA_classif
    elif data_optim.empty is False:
      data_optim = data_optim.merge(PCA_classif, how='outer', on='clf_name')

  ## Univariate feature selection (kbest) optimization
  ## thresholds used in gridsearch 
  number_of_features = [5, 10, 25, 50, 100]
  for number in number_of_features:  
    ## Apply kbest, with given threshold, for all different classifiers to be optimized
    Kbest = SelectKBest(f_classif, k=number).fit(x_train, y_train)
    x_Kbest_train = pd.DataFrame(Kbest.transform(x_train))
    
    Kbest_name = f'Univariate {number}'
    Kbest_classif = classif_hyperpar(x_Kbest_train, y_train, Kbest_name)
    if data_optim.empty:
      data_optim = Kbest_classif
    elif data_optim.empty is False:
      data_optim = data_optim.merge(Kbest_classif, how='outer', on='clf_name')

  return data_optim


def gridsearch (data):
  '''
  perform a manual gridsearch to perform model optimization
  '''
  data_train, data_test, labels_train, labels_test = split(data)
  x_train, x_test = delnan(data_train, data_test)
  x_scaled_train, x_scaled_test = standardscaler(x_train, x_test)
  data_optim = features_hyperpar(x_scaled_train, labels_train)
  
  ## print and save results
  pd.set_option("display.max_rows", None, "display.max_columns", None)
  print('Results')
  print(data_optim)
  
  avg = data_optim.mean()
  print('-'*80)
  print(f'Mean performance for each feature selection method:\n{avg}')
  
  maximum = avg.idxmax()
  print('-'*80)
  print(f'Feature selection method with best mean performance: {maximum}')
  
  clf = data_optim[str(maximum)].idxmax()
  value = data_optim[str(maximum)].max()
  print('-'*80)
  print(f'Classifier with best performance using {maximum}:')
  print(f"{data_optim['clf_name'][clf]}: {value}")
  print('-'*80)
  print(f"Below, fill in {maximum} and {data_optim['clf_name'][clf]}")

  ## Save results
  path = Path('optimization_TM10007.csv')
  data_optim.to_csv(path)


In [0]:
'''
Classifier performance evaluation functions
'''
def select_features(x_train, x_test, y_train, parameters):
  '''
  Select discriminating features using PCA & univariate feature selection 
  '''
  ## PCA feature selection
  if parameters[0].upper() == 'PCA':
    pca = PCA(n_components=None)
    pca.fit(x_train)

    variancelist = np.cumsum(pca.explained_variance_ratio_)

    ## Determine the amount of features within the given threshold
    comp = np.searchsorted(variancelist, parameters[1])

    pca_spec = PCA(n_components=comp)
    pca_spec.fit(x_train)
    
    ## Apply PCA to the different sets
    x_feat_selected_train = pd.DataFrame(pca_spec.transform(x_train))
    x_feat_selected_test = pd.DataFrame(pca_spec.transform(x_test))

    
    return x_feat_selected_train, x_feat_selected_test, _
  
  ## Select the best features and apply them to the different sets
  elif parameters[0].upper() == 'UNIVARIATE':
    Kbest = SelectKBest(f_classif, k=parameters[1]).fit(x_train, y_train)
    x_feat_selected_train = pd.DataFrame(Kbest.transform(x_train))
    x_feat_selected_test = pd.DataFrame(Kbest.transform(x_test))

    ## determine the used features
    feature_names = list(x_train.columns.values)
    mask = Kbest.get_support() #list of booleans
    used_features = [] # The list of the best features

    for bool, feature in zip(mask, feature_names):
      if bool:
          used_features.append(feature)

    return x_feat_selected_train, x_feat_selected_test, used_features


def clf_test(x_train, x_test, y_train,  y_test, parameters):
  '''
   Evaluate classifier performance
  '''
  ## RF classifier
  if parameters[2].upper() == 'RF':
    clf = RandomForestClassifier(n_estimators=parameters[3])
    clf.fit(x_train, y_train)
    y_pred = clf.predict_proba(x_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    y_pred_sens = clf.predict(x_test)
    matrix = confusion_matrix(y_test, y_pred_sens)
    sens = matrix[0,0] / (matrix[0,0]+matrix[0,1])
    spec = matrix[1,1] / (matrix[1,0]+matrix[1,1])

    perf = [auc, sens, spec]
    return perf

  ## SVM classifier
  elif parameters[2].upper() == 'SVM':
    clf = svm.SVC(C= parameters[4], kernel= parameters[5], probability=True)
    clf.fit(x_train, y_train)
    y_pred = clf.predict_proba(x_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    y_pred_sens = clf.predict(x_test)
    matrix = confusion_matrix(y_test, y_pred_sens)
    sens = matrix[0,0] / (matrix[0,0]+matrix[0,1])
    spec = matrix[1,1] / (matrix[1,0]+matrix[1,1])

    perf = [auc, sens, spec]
    return perf

  ## KNN classifier
  elif parameters[2].upper() == 'KNN':
    clf = KNeighborsClassifier(n_neighbors=parameters[6])
    clf.fit(x_train, y_train)
    y_pred=clf.predict_proba(x_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    y_pred_sens = clf.predict(x_test)
    matrix = confusion_matrix(y_test, y_pred_sens)
    sens = matrix[0,0] / (matrix[0,0]+matrix[0,1])
    spec = matrix[1,1] / (matrix[1,0]+matrix[1,1])

    perf = [auc, sens, spec]
    return perf


def test_performance(x_train, x_test, y_train, y_test, parameters):
  '''
  perform all steps required to test optimized model performance
  '''

  ## use given scaling method, clf and hyperparameters to test performance
  x_train, x_test = delnan(x_train, x_test)
  x_train, x_test = standardscaler(x_train, x_test)
  x_train, x_test, features = select_features(x_train, x_test, y_train, 
                                             parameters)
  # pair_plot(x_train, y_train, features)
  perf = clf_test(x_train, x_test, y_train, y_test, parameters)

  return clf, perf, features


def start_test(data, iterations, feature_method, feature_threshold, clf, 
                trees, slack, kernel, neighbors):

  iteration = 0
  parameters = [feature_method, feature_threshold, clf, 
                trees, slack, kernel, neighbors]
  outcome = pd.DataFrame(columns=['Area Under the Curve', 'Sensitivity', 
                                  'Specificity'])
  used_features = pd.DataFrame(columns=['Feature'])
  
  ## Run the script the desired amount of iterations
  while iteration < iterations:
    x_train, x_test, y_train, y_test = split(data)
    clf, perf, features = test_performance(x_train, x_test, y_train, 
                                           y_test, parameters)
    
    ## add result of iteration to dataframe
    add_result = {'Area Under the Curve': perf[0],
                  'Sensitivity': perf[1], 'Specificity': perf[2]}
    outcome = outcome.append(add_result, ignore_index=True)
    
    ## add features used in iteration to dataframe
    for feature in features:
      add_feature = {'Feature': feature}
      used_features = used_features.append(add_feature, ignore_index=True)
      
    iteration += 1

  ## Print results
  print_result(outcome, used_features, parameters)

  ## Save results
  path = Path('test_results_TM10007.csv')
  outcome.to_csv(path)


In [0]:
'''
run all steps required for hyperparameter optimization
and print the dataframe containing grid search results.

Use the results in the following steps
'''
gridsearch(data)

Results
           clf_name   PCA 0.5  PCA 0.75   PCA 0.9  PCA 0.95  Univariate 5  \
0             RF 10  0.843068  0.824280  0.770833  0.660985      0.929508   
1             RF 30  0.847500  0.836250  0.801250  0.737083      0.917216   
2            RF 100  0.846193  0.849792  0.823542  0.825549      0.945549   
3            RF 200  0.841136  0.873030  0.844110  0.805227      0.935871   
4    SVM linear 0.3  0.866742  0.891667  0.877652  0.907841      0.914735   
5    SVM linear 0.1  0.862083  0.888258  0.875379  0.907841      0.904356   
6   SVM linear 0.05  0.862159  0.889659  0.887273  0.907841      0.905492   
7       SVM rbf 0.3  0.843636  0.893409  0.897235  0.909924      0.940909   
8       SVM rbf 0.1  0.847273  0.895682  0.897235  0.909924      0.927955   
9      SVM rbf 0.05  0.847273  0.895682  0.897235  0.909924      0.919773   
10     SVM poly 0.3  0.818030  0.837424  0.856970  0.856856      0.896591   
11     SVM poly 0.1  0.830568  0.862576  0.871818  0.876364      0.8

In [0]:
'''
test the optimized classifier "iterations" time

Fill in the feature selection method, classifier and optimized hyperparamaters based on 
the gridsearch above

feature_method = type of feature method ('pca', 'univariate')
feature_threshold = hyperparameter for specific feature selection method
                    when using pca: (0.5, 0.75, 0.9, 0.95)
                    when using univariate: (5, 10, 25, 50, 100)
clf = type of classifier ('RF', 'SVM', 'KNN')
trees = number of trees in RF (10, 30, 100, 200)
slack = slack value in SVM (0.05, 0.1, '0.3)
kernel = kernel in SVM ('rbf', 'poly', 'linear')
neighbors = number of neighbors in KNN (3, 7, 11, 15)

if no value is needed, fill in 0
'''
feature_method = 'univariate'
feature_threshold = 5
clf = 'RF' 
trees = 100
slack = 0
kernel = 0
neighbors = 0

iterations = 10

start_test(data, iterations, feature_method, feature_threshold, clf, trees, slack, kernel, neighbors)

Results--------------------------------------------------------------------------------

   Area Under the Curve  Sensitivity  Specificity
0              0.953571     0.900000     0.857143
1              0.915789     0.842105     0.933333
2              0.926136     0.954545     0.666667
3              0.897917     0.875000     0.800000
4              0.964427     0.956522     0.818182
5              0.911458     0.944444     0.687500
6              0.793561     0.909091     0.750000
7              1.000000     0.954545     1.000000
8              0.950000     0.950000     0.857143
9              0.863636     0.909091     0.583333
Results over all iterations
Mean AUC: 0.9176496113059327 (min: 0.7935606060606062 max: 1.0 )
Sensitivity: 0.91953441740055 (min: 0.8421052631578947 max: 0.9565217391304348 )
Specificity: 0.7953300865800865 (min: 0.5833333333333334 max: 1.0 )
Prevalence of selected features:
--------------------------------------------------------------------------------

VOLU

In [0]:
def crosval (x_train, y_train, x_test, y_test):
  '''
   Determine classifier performance using cross-validation
  '''
  ## Identify the classifiers used for cross validation
  clfs = [svm.SVC(C=0.05 ,kernel='linear'), KNeighborsClassifier(n_neighbors=9), RandomForestClassifier(n_estimators=5, random_state=42, max_depth=4)]
  clfs_name = ['Support Vector Machine','K Nearest Neighbors','Random Forest']

  ## Start cross validation
  # num = 0
  train_acc = []

  for clf in clfs:
    ## Plot learning curve
    # fig = plt.figure(figsize=(24,8*len(clfs)))
    # ax=fig.add_subplot(4, 3, num + 1)
    # num += 1
    # plot_learning_curve(clf,str(type(clf)), x_train, y_train, axes=ax)

    ## Calculate performance
    scores = cross_val_score(clf, x_train, y_train, cv=5)
    mean_scores = scores.mean()
    train_acc.append(mean_scores)

  ## Determine the best performing classifier
  max_score = max(train_acc)
  array = np.array(train_acc)
  idx_max = np.argmax(array)
  clf_max = clfs[idx_max]
  clf_max_name = clfs_name[idx_max]

  ## Apply chosen classifier on test data
  clf_max.fit(x_train, y_train)
  y_pred=clf_max.predict(x_test)
  accuracy = accuracy_score(y_test, y_pred)
  matrix = confusion_matrix(y_test, y_pred)
  sens = matrix[0,0] / (matrix[0,0]+matrix[0,1])
  spec = matrix[1,1] / (matrix[1,0]+matrix[1,1])

  perf = [accuracy, sens, spec]

  return clf_max_name, perf


def start(data, iterations):
  '''
  Run the iterations. Print and save results.
  '''
  ## Run the script "iterations" time
  iteration = 0
  outcome = pd.DataFrame(columns=['Classifier', 'Accuracy','Sensitivity','Specificity'])
  used_features = pd.DataFrame(columns=['Feature'])
  while iteration < iterations:
    x_train, x_test, y_train, y_test = split(data)
    clf, perf, features = runscript(x_train, x_test, y_train, y_test)
    
    ## add results of iteration to dataframe
    add_result = {'Classifier': str((clf)), 'Accuracy': perf[0],
                  'Sensitivity': perf[1], 'Specificity': perf[2]}
    outcome = outcome.append(add_result, ignore_index=True)
    
    ## add features used in iteration to dataframe
    for feature in features:
      add_feature = {'Feature': feature}
      used_features = used_features.append(add_feature, ignore_index=True)
      
    iteration += 1
  
  ## Print results
  print_result(outcome, used_features)
  
  ## Save results
  path = Path('results_TM10007.csv')
  outcome.to_csv(path)
  
  return outcome